In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import pca
from sklearn.metrics import (f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, precision_recall_curve, 
                             average_precision_score, classification_report, accuracy_score)
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.neural_network import MLPClassifier

from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler


from sklearn.preprocessing import MinMaxScaler
import itertools

C:\Users\cristiano.melo\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Performance Metrics

In [2]:
def get_scores(y_test, y_pred):
    scores = []
    
    scores.append(f1_score(y_test, y_pred, average='micro'))
    print("F1-Score(micro): " + str(scores[-1]))
    
    scores.append(f1_score(y_test, y_pred, average='macro'))
    print("F1-Score(macro): " + str(scores[-1]))
    
    scores.append(f1_score(y_test, y_pred, average='weighted'))
    print("F1-Score(weighted): " + str(scores[-1]))
    
    scores.append(f1_score(y_test, y_pred, average=None))
    print("F1-Score(None): " + str(scores[-1]))
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    #ACC
    scores.append(accuracy_score(y_test, y_pred, normalize=True))
    print("Accuracy: " + str(scores[-1]))
    
    #Sensitivity
    sensitivity = tp / (tp+fn)
    scores.append(tp / (tp+fn))
    print("Sensitivity: " + str(scores[-1]))
    
    #Specificity
    specificity = tn / (tn+fp)
    scores.append (tn / (tn+fp))
    print("Specificity: " + str(scores[-1]))
    
    #VPP
    scores.append(tp / (tp+fp))
    #print("VPP: " + str(scores[-1]))
    
    #VPN
    scores.append(tn / (tn+fn))
    #print("VPN: " + str(scores[-1]))
    
    #RVP
    scores.append(sensitivity / (1-specificity))
    #print("RVP: " + str(scores[-1]))
    
    #RVN
    scores.append((1 - sensitivity) / specificity)
    #print("RVN: " + str(scores[-1]))
    
    #Confusion Matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print("Confusion Matrix: [" + str(cnf_matrix[0][0]) + ", " + str(round(cnf_matrix[1][1],2)) + "]")
    
    #ROC_AUC
    scores.append(roc_auc_score(y_test, y_pred))
    print("ROC AUC score: " + str(scores[-1]))
        
    scores.append([tn, fp, fn, tp])
    
    return scores

In [3]:
def plot_AUC(xv1,yv1):
    proba = lr.predict_proba(xvl)[:,1]
    frp,trp, threshold = roc_curve(yvl,proba)
    roc_auc_ = auc(frp,trp)

    plt.figure(figsize=(5,5))
    plt.title('Reciever Operating Characteristics')
    plt.plot(frp,trp,'r',label = 'AUC = %0.2f' % roc_auc_)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'b--')
    plt.ylabel('True positive rate')
    plt.xlabel('False positive rate')

In [4]:
def plot_confusion_matrix(cm,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    #    print("Normalized confusion matrix")
    #else:
    #    print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    #tick_marks = np.arange(len(classes))
    #plt.xticks(tick_marks, classes, rotation=45)
    #plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def plot_confusion_matrixes(y_test, y_pred):
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    #plt.figure()
    #plt.subplots(1,2,figsize=(20,4))
    #plt.subplot(1,2,1)
    #plot_confusion_matrix(cnf_matrix, title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    #plt.subplot(1,2,2)
    plot_confusion_matrix(cnf_matrix, normalize=True, title='Normalized confusion matrix')

    plt.tight_layout()
    plt.show()

# Outliers Removal

In [5]:
def outliers_counter(df):

    outliers_index_list = list()

    for index, row in df.iterrows():
        outlier_counter = 0
        if row.CountClassCoupled > boxplot_max_list[0]:
            outlier_counter = outlier_counter + 1

        if row.SumCyclomatic > boxplot_max_list[1]:
            outlier_counter = outlier_counter + 1

        if row.MaxInheritanceTree > boxplot_max_list[2]:
            outlier_counter = outlier_counter + 1

        if row.PercentLackOfCohesion > boxplot_max_list[3]:
            outlier_counter = outlier_counter + 1

        if row.CountLineCode > boxplot_max_list[4]:
            outlier_counter = outlier_counter + 1

        if row.CountClassDerived > boxplot_max_list[5]:
            outlier_counter = outlier_counter + 1

        if row.CountDeclMethodAll > boxplot_max_list[6]:
            outlier_counter = outlier_counter + 1

        if row.CountDeclMethod > boxplot_max_list[7]:
            outlier_counter = outlier_counter + 1

        if outlier_counter >= 4:
            outliers_index_list.append(index)
    
    return outliers_index_list

# Algorithms

In [6]:
def LogisticRegr_(Xtrain, Ytrain, Xtest, Ytest):
    print("\nLOGISTIC REGRESSION")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_LR, xvl_LR = X_train.iloc[train_index], X_train.iloc[test_index]
        ytr_LR, yvl_LR = y_train.iloc[train_index], y_train.iloc[test_index]

        #model
        lr = LogisticRegression(solver='lbfgs', random_state=42, class_weight='balanced')
        lr.fit(xtr_LR, ytr_LR.values.ravel())
        score = roc_auc_score(yvl_LR, lr.predict(xvl_LR))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))

    print("\nTEST SET:")
    get_scores(Ytest, lr.predict(Xtest))

In [7]:
def DecisionTree_(Xtrain, Ytrain, Xtest, Ytest):
    %%time
    print("\nDECISION TREE")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_DT, xvl_DT = X_train.iloc[train_index], X_train.iloc[test_index]
        ytr_DT, yvl_DT = y_train.iloc[train_index], y_train.iloc[test_index]

        #model
        dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')
        dt.fit(xtr_DT, ytr_DT.values.ravel())
        score = roc_auc_score(yvl_DT, dt.predict(xvl_DT))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1


    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))    

    print("\nTEST SET:")
    get_scores(Ytest, dt.predict(Xtest))

In [8]:
def RandomForest_(Xtrain, Ytrain, Xtest, Ytest):
    %%time
    print("RANDOM FOREST")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_RF, xvl_RF = X_train.iloc[train_index], X_train.iloc[test_index]
        ytr_RF, yvl_RF = y_train.iloc[train_index], y_train.iloc[test_index]

        #model
        rf = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)
        rf.fit(xtr_RF, ytr_RF.values.ravel())
        score = roc_auc_score(yvl_RF, rf.predict(xvl_RF))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))    

    print("\nTEST SET:")
    get_scores(Ytest, rf.predict(Xtest))

In [9]:
def NN_(Xtrain, Ytrain, Xtest, Ytest):
    %%time

    print("NEURAL NETWORK")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_NN, xvl_NN = X_train.iloc[train_index], X_train.iloc[test_index]
        ytr_NN, yvl_NN = y_train.iloc[train_index], y_train.iloc[test_index]

        #model
        nn = MLPClassifier(random_state=42)
        nn.fit(xtr_NN, ytr_NN.values.ravel())
        score = roc_auc_score(yvl_NN, nn.predict(xvl_NN))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))   

    print("\nTEST SET:")
    get_scores(Ytest, nn.predict(Xtest))

# Algorithms (no iloc)

In [10]:
def LogisticRegr_NoIloc(Xtrain, Ytrain, Xtest, Ytest):
    %%time
    print("\nLOGISTIC REGRESSION")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_LR, xvl_LR = Xtrain[train_index], Xtrain[test_index]
        ytr_LR, yvl_LR = Ytrain[train_index], Ytrain[test_index]

        #model
        lr = LogisticRegression(solver='lbfgs', random_state=42, class_weight='balanced')
        lr.fit(xtr_LR, ytr_LR)
        score = roc_auc_score(yvl_LR, lr.predict(xvl_LR))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))

    print("\nTEST SET:")
    get_scores(Ytest, lr.predict(Xtest))

In [11]:
def DecisionTree_NoIloc(Xtrain, Ytrain, Xtest, Ytest):
    %%time
    print("\nDECISION TREE")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_DT, xvl_DT = Xtrain[train_index], Xtrain[test_index]
        ytr_DT, yvl_DT = Ytrain[train_index], Ytrain[test_index]
        #model
        dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')
        dt.fit(xtr_DT, ytr_DT)
        score = roc_auc_score(yvl_DT, dt.predict(xvl_DT))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))    

    print("\nTEST SET:")
    get_scores(Ytest, dt.predict(Xtest))

In [12]:
def RandomForest_NoIloc(Xtrain, Ytrain, Xtest, Ytest):
    %%time
    print("RANDOM FOREST")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_RF, xvl_RF = Xtrain[train_index], Xtrain[test_index]
        ytr_RF, yvl_RF = Ytrain[train_index], Ytrain[test_index]

        #model
        rf = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)
        rf.fit(xtr_RF, ytr_RF)
        score = roc_auc_score(yvl_RF, rf.predict(xvl_RF))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))    

    print("\nTEST SET:")
    get_scores(Ytest, rf.predict(Xtest))

In [13]:
def NN_NoIloc(Xtrain, Ytrain, Xtest, Ytest):
    %%time

    print("NEURAL NETWORK")
    cv_score = []
    i = 1
    print("TRAIN AND VALIDATION SETS:")
    for train_index, test_index in kf.split(Xtrain, Ytrain):
        print('{} of KFold {}'.format(i,kf.n_splits))
        xtr_NN, xvl_NN = Xtrain[train_index], Xtrain[test_index]
        ytr_NN, yvl_NN = Ytrain[train_index], Ytrain[test_index]

        #model
        nn = MLPClassifier(random_state=42)
        nn.fit(xtr_NN, ytr_NN)
        score = roc_auc_score(yvl_NN, nn.predict(xvl_NN))
        print('ROC AUC score:',score)
        cv_score.append(score)    
        i+=1

    print('\nCROSS VALIDANTION SUMMARY:')
    print('Mean: ' + str(np.mean(cv_score)))
    print('Std deviation: ' + str(np.std(cv_score)))   

    print("\nTEST SET:")
    get_scores(Ytest, nn.predict(Xtest))